<a href="https://colab.research.google.com/github/basisvectors/Scene-Change-Detection/blob/main/Scene_Change_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scenedetect[opencv] --upgrade --quiet
!pip install ffmpeg --quiet
!pip install -U yt-dlp --quiet
!apt-get -qq install mkvtoolnix

!git clone https://github.com/slhck/bufferer.git
!pip install bufferer

import shutil
import os

from IPython.display import display
from ipywidgets import Video

# Standard PySceneDetect imports:
from scenedetect import VideoManager
from scenedetect import SceneManager

# For content-aware scene detection:
from scenedetect.detectors import ContentDetector


In [ ]:
#@markdown #Video Downloder{ vertical-output: true }

def find_scenes(video_path, threshold=30.0):
    # Create our video & scene managers, then add the detector.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    # Improve processing speed by downscaling before processing.
    video_manager.set_downscale_factor()

    # Start the video manager and perform the scene detection.
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    # Each returned scene is a tuple of the (start, end) timecode.
    scene_list = scene_manager.get_scene_list()
    # print(len(scene_list))
    splits = ""
    for i in scene_list:
      splits += f"[{str(i[1].get_seconds())},{DURATION}],"
    splits = splits[:-1]

    return scene_list, splits

url = "https://www.youtube.com/watch?v=ZVjo86OEeFc" #@param {type:"string"}
id = "/content/" + url.split("=")[1]

!mkdir {id}

filepath = f"{id}/input.mp4"
!yt-dlp {url} -o {filepath} -f mp4
#@markdown ### Scene change pause duration
DURATION = 1 #@param {type:"number"}

In [ ]:
scene_list, splits = find_scenes(filepath)
print(splits)

100%|██████████| 720/720 [00:05<00:00, 142.69frames/s]

[2.8361666666666663,1],[5.588916666666666,1],[6.5065,1],[8.133125,1],[9.759749999999999,1],[11.011,1],[12.429083333333333,1],[13.805458333333332,1],[15.014999999999999,1],[15.682333333333332,1],[17.100416666666664,1],[18.143124999999998,1],[18.935583333333334,1],[19.894875,1],[20.812458333333332,1],[22.230541666666664,1],[22.897875,1],[23.732041666666664,1],[26.276249999999997,1],[27.861166666666666,1],[29.195833333333333,1],[30.029999999999998,1]


In [ ]:
#@title Preview { vertical-output: true }

spinner = "./bufferer/spinners/spinner-256-white.png" #@param ["./bufferer/spinners/spinner-256-white.png", "./bufferer/spinners/spinner-256-black.png"]

!bufferer -i {filepath} -b {splits} -o output.mp4 -v libx264 -a aac --verbose -s {spinner}

Video.from_file("/content/output.mp4")